# Version 1 (Base)

In [1]:
import cv2
from pyzbar import pyzbar
def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        print(barcode_info)
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2.0, (255, 255, 255), 1)
        with open("barcode_result.txt", mode ='w') as file:
            file.write("Recognized Barcode:" + barcode_info)
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('Barcode/QR code reader', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
            
    camera.release()
    cv2.destroyAllWindows()
if __name__ == '__main__':
    main()

C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757
C1K20105757


# Version 2 (No rectangles)

In [ ]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep

def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        barcode_info = barcode.data.decode('utf-8')
        print(barcode_info)
        with open("barcode_result.txt", mode ='a') as file:
            current_datetime = dt.datetime.now()
            string_date = current_datetime.strftime(" %d %B %Y, %A, %I:%M:%S\n")
            file.write(barcode_info + string_date)
        sleep(5)
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('Barcode/QR code reader', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break        
    camera.release()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

# Version 3 (multithreading)

In [ ]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep
import time
import threading

def file_operation(barcode_info):
    with open("barcode_result.txt", mode ='a') as file:
            current_datetime = dt.datetime.now()
            string_date = current_datetime.strftime(" %d %B %Y, %A, %I:%M:%S\n")
            file.write(barcode_info + string_date)
            print(barcode_info)
            time.sleep(3)
            
def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        #print(barcode_info)
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2.0, (255, 255, 255), 1)
        file_operation_thread = threading.Thread(target=file_operation, args=(barcode_info,))
        file_operation_thread.start()
        file_operation_thread.join()
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('Barcode/QR code reader', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break        
    camera.release()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

# Version 4 (Last Check + rectangles) 

In [ ]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep
import time
import threading
from datetime import timedelta
from datetime import datetime

def check_last():
    with open("barcode_result.txt", mode ='r') as file:
        lines = file.read().splitlines()
    last_line = lines[-1]
    date_time = last_line[12:]
    date_time_obj = datetime.strptime(date_time, "%d %B %Y, %A, %H:%M:%S")
    date_time_now = dt.datetime.now()
    time = date_time_now - date_time_obj
    file.close()
    return time.total_seconds()

def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        cv2.rectangle(frame, (x, y),(x+w, y+h), (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2.0, (255, 255, 255), 1)
        last_time = check_last()
        if(last_time > 10.0):
            with open("barcode_result.txt", mode ='a') as file:
                current_datetime = dt.datetime.now()
                string_date = current_datetime.strftime(" %d %B %Y, %A, %H:%M:%S\n")
                file.write(barcode_info + string_date)
            file.close()
            print(barcode_info)
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('PBL Project', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break        
    camera.release()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

# Version 5 (Last Check + rectangles + Time Constraints + Validity) 

In [3]:
import cv2
import datetime as dt
from pyzbar import pyzbar
from time import sleep
import time
import threading
from datetime import timedelta
from datetime import datetime

def check_validity(barcode_data):
    if(len(barcode_data) == 11):
        if(barcode_data[2] == "K"):
            if(barcode_data[0:3] in ["C2K", "E2K", "I2K", "C1K", "E1K"]):
                return True
            else:
                return False
        else:
            return False
    else:
        return False

def check_last():
    with open("barcode_result.txt", mode ='r') as file:
        lines = file.read().splitlines()
    last_line = lines[-1]
    date_time = last_line[12:]
    date_time_obj = datetime.strptime(date_time, "%d %B %Y, %A, %H:%M:%S")
    date_time_now = dt.datetime.now()
    time = date_time_now - date_time_obj
    file.close()
    return time.total_seconds()

def check_same(to_check):
    with open("barcode_result.txt", mode ='r') as file:
        lines = file.read().splitlines()
    last_line = lines[-1]
    last_check = last_line[:11]
    if(to_check == last_check):
        return True

def read_barcode(frame):
    barcodes = pyzbar.decode(frame)
    for barcode in barcodes:
        font = cv2.FONT_HERSHEY_DUPLEX
        x, y , w, h = barcode.rect
        barcode_info = barcode.data.decode('utf-8')
        last_time = check_last()
        cv2.rectangle(frame, (x, y),(x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, barcode_info, (x + 6, y - 6), font, 2, (255, 255, 255), 1)
        if(check_validity(barcode_info)):
            if((last_time > 5.0) and not (check_same(barcode_info) and (last_time < 100.0))):
                with open("barcode_result.txt", mode ='a') as file:
                    current_datetime = dt.datetime.now()
                    string_date = current_datetime.strftime(" %d %B %Y, %A, %H:%M:%S\n")
                    file.write(barcode_info + string_date)
                file.close()
                print(barcode_info)
            elif(check_same(barcode_info) and (last_time < 100.0)):
                cv2.putText(frame, "ATTENDANCE LOGGED", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
            else:
                cv2.putText(frame, "PLEASE WAIT A FEW SECONDS", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
        else:
            cv2.putText(frame, "WRONG BARCODE", (x - 100, y + 100), font, 1, (0, 0, 255), 1)
    return frame

def main():
    camera = cv2.VideoCapture(1)
    ret, frame = camera.read()
    while ret:
        ret, frame = camera.read()
        frame = read_barcode(frame)
        cv2.imshow('PBL Project', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break        
    camera.release()
    cv2.destroyAllWindows()
    
if __name__ == '__main__':
    main()

E2K20103958
